<a href="https://colab.research.google.com/github/Zafar-Beg/SnowHydrology/blob/main/GlacierMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/*
 * Code for the manuscript "From white to green: Multidecadal trends of
 * snow cover and vegetation productivity in the European Alps" by
 * Rumpf et al., submitted December 2021 to Science
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU General Public License as published by
 * the Free Software Foundation, version 3.
 *
 * This program is distributed in the hope that it will be useful, but
 * WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
 * General Public License for more details.
 *
 * You should have received a copy of the GNU General Public License
 * along with this program. If not, see <http://www.gnu.org/licenses/>.
 */

var projectPath=// <--- you need to set the project path here
var glacier = ee.FeatureCollection("GLIMS/current"),
    geometry =
    /* color: #d63000 */
    /* shown: false */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */
    ee.Geometry.Polygon(
        [[[5.501897369938105, 48.198582724702206],
          [5.501897369938105, 43.36861058284113],
          [15.114934479313105, 43.36861058284113],
          [15.114934479313105, 48.198582724702206]]], null, false);
/*
compute the first observation of no glacier
*/
// a good aproximation of the galcier bounds
// not really important just to have an export area
var glacierBounds=glacier.filterBounds(geometry).randomColumn('noise').sort('noise').limit(10000)

// buffer it because it's an approximation
var containedgeometry=ee.Feature(ee.FeatureCollection(glacierBounds.map(function(ft){return ft.bounds()})).union(1000).first()).convexHull().geometry().buffer(1000);
Map.addLayer(containedgeometry)


//for each geometry compute the inetrescting galcier, the glacier id is used for this task
// sort by yers to get the first non obeserved galcier only pervious time are youse full,
// we take the closest in tame with teh same id
glacier=glacier.map(function(ft){return ft.set('year',ee.Number.parse(ee.String(ft.get('src_date')).slice(0,4)));});
//var glacierMapLG=glacier.sort('year',false).reduceToImage(['year'], ee.Reducer.firstNonNull());
var glac_if = ee.Filter.and(ee.Filter.maxDifference({
 difference: 10000,// not importnat, but must be big enough (>200)
  leftField: 'year',
  rightField: 'year'
}),
ee.Filter.lessThan({
  leftField:'year',
  rightField:'year'
}),
ee.Filter.equals({
  leftField:'glac_id',
  rightField:'glac_id'
}));

var saveBestJoin = ee.Join.saveBest({
  matchKey: 'nextGlacierRecord',
  measureKey: 'timeDiff'
});

// Apply the join.
var intersectJoined = saveBestJoin.apply(glacier, glacier, glac_if);
intersectJoined=intersectJoined.map(function(ft){return ft.set('follwingObservationYear',ee.Feature(ft.get('nextGlacierRecord')).get('year'))});

var firstNoIceObservation=intersectJoined.sort('year',false).reduceToImage(['follwingObservationYear'], ee.Reducer.firstNonNull());


//stack all the interesection to get a single map
var lastIceObservation=glacier.reduceToImage(['year'], ee.Reducer.max());


var image=firstNoIceObservation.rename('fNIO').addBands(lastIceObservation.rename('lIO'));

Map.addLayer(image,{min: 1950, max:2020});
// export as asset
Export.image.toAsset({
    image:image,
    description:'export_firstNoIceObservation',
    assetId:projectPath+'IceObservation',
    pyramidingPolicy:'MAX',
//	dimensions:,
    region:containedgeometry,
    scale:30/2,
//	crs:,
//	crsTransform:,
    maxPixels:5e9,
//	shardSize:,
})

